<a href="https://www.kaggle.com/code/raxmatillo/aviachiptabashorat?scriptVersionId=218893108" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


# Ma'lumotlarni tayyorlab olamiz

In [2]:
train = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv')
test = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv')
sample = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv')

In [3]:
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [4]:
train.isnull().sum()

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [6]:
train.shape

(20000, 12)

In [7]:
train.select_dtypes(include='object').columns

Index(['airline', 'flight', 'source_city', 'departure_time', 'stops',
       'arrival_time', 'destination_city', 'class'],
      dtype='object')

In [8]:
# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder()

# train_cat_1hot = encoder.fit_transform(train[['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']]).toarray()

In [9]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# train_nums = scaler.fit_transform(train[['duration', 'days_left']])

# Machine Learningga tayyorlaymiz

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X = train.drop('price', axis=1)
y = train['price']

scaler = StandardScaler()

nums = ['duration', 'days_left']
cats = ['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')


full_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_transformer, nums),  # Raqamli ustunlarni o'zgartirish
        ('cat', cat_transformer, cats)   # Kategorik ustunlarni o'zgartirish
    ])

# df = pd.get_dummies(df, columns=cats)
X_prepared = full_pipeline.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X_prepared, y, test_size=0.2, random_state=42)

X_train.shape

(16000, 1347)

# Baholash funksiyamiz

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

def score(y_train, y_pred):
    mae = mean_absolute_error(y_train, y_pred)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    # EVS hisoblash
    evs = explained_variance_score(y_train, y_pred)
    print(f'MAE: {mae}')
    print(f'RMSE: {np.sqrt(mse)}')
    print(f'R-squared: {r2}')
    print(f"Explained Variance Score: {evs}")

# Bir nechta regressiya algoritmlari bilan model yozamiz

In [12]:
# LinearRegression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression().fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_valid)

print('LinearRegression (valid set)\n')
score(y_valid, lr_y_pred)

LinearRegression (valid set)

MAE: 4376.5665032694615
RMSE: 6334.636572663727
R-squared: 0.9235833101917663
Explained Variance Score: 0.9236261418670145


In [13]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor().fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_valid)

print('RandomForestRegressor:\n')
score(y_valid, rf_y_pred)

RandomForestRegressor:

MAE: 1594.5779174583333
RMSE: 3392.1789387391473
R-squared: 0.9780869757033728
Explained Variance Score: 0.978089452151607


In [14]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor().fit(X_train, y_train)
tree_y_pred = tree_model.predict(X_valid)

print('RandomForestRegressor:\n')
score(y_valid, tree_y_pred)

RandomForestRegressor:

MAE: 1780.976125
RMSE: 4417.812578082337
R-squared: 0.9628328390633254
Explained Variance Score: 0.9628366650568632


# Test data bilan predict qilamiz

In [15]:
test_prepared = full_pipeline.transform(test)
test_prepared

<5000x1347 sparse matrix of type '<class 'numpy.float64'>'
	with 49957 stored elements in Compressed Sparse Row format>

In [16]:
y_pred = rf_model.predict(test_prepared)
y_pred

array([51804.69, 64025.31, 23163.2 , ..., 50249.46, 50505.75, 65372.44])

# Natijani sample datasetining price ustuniga saqlaymiz

In [17]:
sample['price'] = y_pred

In [18]:
sample.head()

,id,price
0,1,51804.69
1,2,64025.31
2,3,23163.20
3,4,2953.18
4,5,5841.38


In [19]:
sample.to_csv('aviachiptabashorat.csv', index=False)